In [317]:
import open3d as o3d
import numpy as np
import copy
import matplotlib.pyplot as plt

In [287]:
mesh_path = './models/Aric Board 2.stl'

mesh = o3d.io.read_triangle_mesh(mesh_path)

In [288]:
# Visualizing the mesh.
draw_geoms_list = [mesh]
o3d.visualization.draw_geometries(draw_geoms_list)

In [289]:
print(mesh)

TriangleMesh with 2048425 points and 683718 triangles.


In [290]:
# Computing the normals for the mesh.
mesh.compute_vertex_normals()

# Visualizing the mesh.
draw_geoms_list = [mesh]
o3d.visualization.draw_geometries(draw_geoms_list)

In [291]:
# Creating a mesh of the XYZ axes Cartesian coordinates frame.
# This mesh will show the directions in which the X, Y & Z-axes point,
# and can be overlaid on the 3D mesh to visualize its orientation in
# the Euclidean space.
# X-axis : Red arrow
# Y-axis : Green arrow
# Z-axis : Blue arrow
mesh_coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=30, origin=[0, 0, 0])

# Visualizing the mesh with the coordinate frame to understand the orientation.
draw_geoms_list = [mesh_coord_frame, mesh]
o3d.visualization.draw_geometries(draw_geoms_list)

In [292]:
# Creating a mesh of the XYZ axes Cartesian coordinates frame.
# This mesh will show the directions in which the X, Y & Z-axes point,
# and can be overlaid on the 3D mesh to visualize its orientation in
# the Euclidean space.
# X-axis : Red arrow
# Y-axis : Green arrow
# Z-axis : Blue arrow
mesh_coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=30, origin=[-300, 200, 200])

# Visualizing the mesh with the coordinate frame to understand the orientation.
draw_geoms_list = [mesh_coord_frame, mesh]
o3d.visualization.draw_geometries(draw_geoms_list)

In [293]:
box = mesh.get_minimal_oriented_bounding_box()
box.color = (1,0,0)

o3d.visualization.draw_geometries([mesh, box])

In [294]:
print(box)
print(box.extent)

OrientedBoundingBox: center: (-321.818, 175.307, 184.743), extent: 487.649, 90.2207, 41.9591)
[487.64862295  90.2206845   41.95912127]


In [295]:
print(mesh.get_center())

[-320.66492028  183.06555677  175.37032867]


In [296]:
new_box = o3d.geometry.TriangleMesh.create_box(box.extent[0], box.extent[1], box.extent[2])

o3d.visualization.draw_geometries([new_box, mesh])

In [297]:
mesh_path = './models/Aric Board 2.stl'

mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()

old_center = mesh.get_center()

mesh = mesh.translate(old_center*(-1))

print(mesh.get_center())

[8.98267951e-10 5.50072564e-10 4.32327229e-10]


In [298]:
box = mesh.get_minimal_oriented_bounding_box()
rect_mesh = o3d.geometry.TriangleMesh.create_box(box.extent[0], box.extent[1], box.extent[2])
mesh_coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=30, origin=[-1, 0, 0])
box_mesh_coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=30, origin=[0, 0, 0])
o3d.visualization.draw_geometries([new_box, mesh, mesh_coord_frame, box_mesh_coord_frame])

# mesh of rect_mesh is positioned so that bottom right corner is (0,0,0). while original mesh is not. need to find a way to position original mesh into the position!!

In [299]:
print(mesh)
box = mesh.get_minimal_oriented_bounding_box()
print(box)

TriangleMesh with 2048425 points and 683718 triangles.
OrientedBoundingBox: center: (-1.15279, -7.75875, 9.37264), extent: 487.649, 90.2207, 41.9591)


ASSUMPTION: model is calibrated so that axis fits perfectly in bottom right hand corner - I am going to manually translate+rotate mesh until fits as such. This is because when using the create_box function, the mesh is always positioned so the bottom right corner is (0,0,0)

In [300]:
o3d.visualization.draw_geometries([mesh, mesh_coord_frame])
mesh_r_old = copy.deepcopy(mesh)

In [301]:
#Rotation check!!!
# Not working
mesh_r = copy.deepcopy(mesh)
R = mesh.get_rotation_matrix_from_xyz((0, 2*np.pi/3 , 0))
mesh_r.rotate(R, center=(0,0,0))
mesh_r.paint_uniform_color([1, 0.706, 0])
box_mesh_coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100, origin=[0, 0, 0])
o3d.visualization.draw_geometries([mesh_r, box_mesh_coord_frame, mesh, rect_mesh])



new approach: 

find the center of mass of the vertices and then translate the whole mesh by the center of mass (edited) 
They might have a function to do that for you but maybe not
once it's translated, apply rotation and then translate it back to where it was if you want it  there

In [302]:
mesh_path = './models/Aric Board 2.stl'
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()
old_center = mesh.get_center()
mesh = mesh.translate(old_center*(-1))


old_center = rect_mesh.get_center()
rect_mesh = rect_mesh.translate(old_center*(-1))
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100, origin=[0, 0, 0])

o3d.visualization.draw_geometries([coord_frame, mesh, rect_mesh])

In [303]:
mesh_copy = copy.deepcopy(mesh)
mesh_copy.rotate(mesh_copy.get_rotation_matrix_from_xyz((np.pi / 2, 0, np.pi / 4)), center=(0, 0, 0))
mesh_copy.paint_uniform_color([1, 0.706, 0])

o3d.visualization.draw_geometries([coord_frame, mesh, rect_mesh, mesh_copy])

In [304]:
# source: https://stackoverflow.com/questions/70160183/how-can-i-align-register-two-meshes-in-open3d-python
#Does not WOrK!!!

#Since it is alot easier to adjust via blender, will save the each centered object, rotate manually, and reimport back to this notebook

o3d.io.write_triangle_mesh("defective_mesh.stl", mesh)
rect_mesh.compute_vertex_normals()

o3d.io.write_triangle_mesh("rect_mesh.stl", rect_mesh)


True

Adjustments made:

Location X: -11.46 m
Y: 2.85 m
Z: 1.43m

Rotation X: 60.4 deg
Y: 5.1 deg
Z: 84.6 deg

Scale X: 1
Y: 1
Z: 1

In [308]:
rect_mesh = o3d.io.read_triangle_mesh("rect_mesh_aligned.stl")
rect_mesh.compute_vertex_normals()
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100, origin=[0, 0, 0])

o3d.visualization.draw_geometries([rect_mesh, coord_frame])

def_mesh = o3d.io.read_triangle_mesh("def_mesh_aligned.stl")
def_mesh.compute_vertex_normals()
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100, origin=[0, 0, 0])

o3d.visualization.draw_geometries([def_mesh, coord_frame])

In [309]:
rect_cloud = rect_mesh.sample_points_uniformly(1000)
def_cloud = def_mesh.sample_points_uniformly(1000)

In [310]:
#https://github.com/isl-org/Open3D/pull/1884/commits/d739cc20e4f54798aacca5f32b36f5a3066531a6
dists = rect_cloud.compute_point_cloud_distance(def_cloud)
dists = np.asarray(dists)
ind = np.where(dists > 0.01)[0]
res = rect_cloud.select_by_index(ind)
o3d.visualization.draw_geometries([res])

Not just the wane - try scaling??

In [313]:
rect_mesh_c = copy.deepcopy(rect_mesh)
rect_mesh_c.scale(0.98, center=rect_mesh_c.get_center())

o3d.visualization.draw_geometries([rect_mesh_c, def_mesh])

In [332]:
rect_cloud = rect_mesh_c.sample_points_uniformly(1000)
def_cloud = def_mesh.sample_points_uniformly(1000)

dists = def_cloud.compute_point_cloud_distance(rect_cloud)
dists = np.asarray(dists)
ind = np.where(dists > 0.01)[0]
res = rect_cloud.select_by_index(ind)
o3d.visualization.draw_geometries([res])

In [333]:
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    res, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([rec_mesh])

[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh

IndexError: _Map_base::at

In [323]:
rect_cloud = rect_mesh_c.sample_points_uniformly(1000)
def_cloud = def_mesh.sample_points_uniformly(1000)

res = rect_cloud.crop_point_cloud(def_cloud)
o3d.visualization.draw_geometries([res])

AttributeError: 'open3d.cpu.pybind.geometry.PointCloud' object has no attribute 'crop_point_cloud'